In [4]:
from dotenv import load_dotenv
load_dotenv()

from pprint import pprint

In [89]:
from langchain.agents import Tool
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS, VectorStore

from typing import List, Union


class SiteSearchGenerator:
    DB_KEY = 'SiteSearchDbKey'
    db: Union[VectorStore, None] = None

    def __init__(
            self, sites: List[str] = [],
            results:int = 10, debug:bool = False,
            embeddings = OpenAIEmbeddings()
        ) -> None:
        self.sites = sites
        self.results = results
        self.debug = debug
        self.embeddings = embeddings
    
    def create_sited_search(self):
        def sited_search(query):
            site_queries = list(map(
                    lambda site_str: f'site:{site_str}',
                    sites
                    ))
            joined_site_query = ' OR '.join(site_queries)

            search_query = query + ' ' + joined_site_query

            if self.debug:
                pprint('searching...')
                pprint(f"Search: {search_query}")

            search_result = DuckDuckGoSearchAPIWrapper().results(search_query, num_results=self.results)

            urls = []
            for r in search_result:
                if r['link']:
                    urls.append(r['link'])
            
            self.embedding_results(urls=urls)

            return search_result
        
        return sited_search

    def embedding_results(self, urls: List[str]) -> List[str]:
        if self.debug:
            pprint('result saving...')
        
        for url in urls:
            if self.debug:
                pprint(f'splitting {url}')
            text_splitter = TokenTextSplitter(
                chunk_size = 100,
                chunk_overlap  = 10,
            )
            loader = WebBaseLoader(url)
            # docs = loader.load()
            # texts = text_splitter.create_documents(self.documents_to_strlist(docs))
            doc = ' '.join(loader.scrape().find('main').get_text())

            texts = text_splitter.create_documents([doc])
            
            if self.debug:
                pprint(f'save vector {url}')

            db = FAISS.from_documents(texts, self.embeddings)
            if self.db == None:
                self.db = db
            else:
                self.db.merge_from(db)

    
    def lookup_db(self, query: str) -> str:
            retriever = self.db.as_retriever()
            docs = retriever.get_relevant_documents(query)
            return "\n\n".join(self.documents_to_strlist(docs))

    def build_cunks(self, texts: List[str]):
        return ''
    
    def documents_to_strlist(self, documents)->List[str]:
        return list(map(lambda d: d.page_content, documents))

sites = [
    'https://jinjibu.jp/',
    'https://hrnote.jp/'
]


c = SiteSearchGenerator(sites=sites, debug=True)


search_tool = Tool(
    name='Search',
    description=(
        # "A wrapper around Duck Duck Go Search. "
        # "Useful for when you need to answer questions about current events. "
        # "Input should be a search query. Output is a JSON array of the query results"
        "DuckDuckGoによる人事労務関連サイトに絞った検索"
        "人事労務に関する専門知識を検索するために有効に使える"
        "Input should be a search query. Output is a JSON array of the query results"
    ),
    func=c.create_sited_search()
)

lookup_tool = Tool(
    name='Lookup',
    description=(
        "searchを行ってヒットしたサイトのEmbeddingから文章を読み込む"
        "人事労務関連の文書をLookupしたいときにつかえる"
        "Input should be a word or sentence. Output is a parsed String"
    ),
    func=c.lookup_db,
)


In [90]:
from langchain import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

tools = [search_tool, lookup_tool]

llm = OpenAI(temperature=0, model_name="gpt-4")
# agent_executor = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

prefix = """あなたは人事労務の専門家です。あなたはできるだけ資料に即して質問に回答します。 You have access to the following tools:"""
suffix = """ "Args: "を用いるのを忘れないようにして下さい。

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [91]:
# question = "I am a human resources manager working for a Japanese company. I would like to '退職' and would like to know exactly how to do so."
question = "わたしは日本の会社で働く人事労務担当者です。退職に関連する業務の洗い出しを行いたいのですが、詳細がわからないのでおしえてください。"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: 退職に関連する業務の洗い出しについて調べる。
Action: Search
Action Input: 退職に関連する業務 日本の会社 人事労務'searching...'
('Search: 退職に関連する業務 日本の会社 人事労務 site:https://jinjibu.jp/ OR '
 'site:https://hrnote.jp/')


/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-package

'result saving...'
'splitting https://hrnote.jp/contents/roumu-rodokijunho-taishoku-20230116/'
('\n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' 労 働 基 準 法 に お け る 退 職 の 定 義 は ？ 手 続 き 方 法 や 注 意 点 を 紹 介 \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' 労 務   \n'
 ' \n'
 ' 労 務 ・ そ の 他   \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' 2 0 2 3 . 0 6 . 0 2                                                         '
 '最 終 更 新 \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 ' 退 職 に 関 す る こ と は 、 労 働 基 準 法 に よ っ て 定 め ら れ て い ま す 。 し か し 、 民 法 や 就 業 規 '
 '則 な ど で も 退 職 に つ い て 触 れ ら れ て い る た め 、 ど の 定 義 を 確 認 す れ ば 良 い か わ か ら な '
 'い こ と も あ る で し ょ う 。 \n'
 ' 本 記 事 で は 、 労 働 基 準 法 に お け る 退 職 の 定 義 に つ い て 解 説 し ま す 。 さ ら に 、 人 事 担 当 '
 '者 が 行 う べ き 退 職 手 続 き の 方 法 や 注 意 点 に つ い て も 紹 介 す る の で 、 日 々 の 業 務 に お 役 '
 '立 て く だ さ い 。 \n'
 ' 法 改 正 か ら 基 本 的 な 内 容 ま で 分 か り や す く 解 説 ！ \n'
 ' 労 働 基 準 法 総 ま と め B O O K \n'
 ' \n'
 ' 労 働 基 準 法 の 内 容 を 詳 細 に 把 握 し て い ま す か 

OutputParserException: Could not parse LLM output: `検索結果から、退職に関連する業務には、労働基準法に基づく退職手続き、保険に関する手続き（雇用保険、健康保険、厚生年金、住民税）、離職票と離職証明書の発行などが含まれます。具体的な手続きについては、従業員が退職する際に総務担当者が行う手続きが多岐にわたりますので、退職日までにするべきことや、その後に行う手続きについて確認しましょう。また、退職金の支払い義務や金額の決め方については、労働基準法には退職金の取り決めがなく、企業側で自由に決めることができます。`

In [67]:
c.lookup_db('退職')

'\n\n\n\n\n\n\n\n\n退職\n\n\n\n\n\n\n\n\n\n\n退職\n\nいます。「退職\n\n退職！�'